<a href="https://colab.research.google.com/github/dangthanhkhanhdan962-ops/AI_sercurity/blob/main/AIS_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install websockets==12.0

In [ ]:
import os
os.environ["AISSTREAM_API_KEY"] = "772c01c9ee72d443288acb609635f09b2748f8b7"  # thay bằng key của bạn

**Cho nó cào dữ liệu 13 cái cảng chỉ định kèm chỉ lấy tàu hạng A **

In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import csv
import json
import time
import asyncio
from datetime import datetime, timezone
import websockets

AIS_URL = "wss://stream.aisstream.io/v0/stream"

# ======= CẤU HÌNH =======
# Bbox cho 13 cảng (lat, lon): góc tây-nam và đông-bắc
PORT_BBOXES = [
    [[10.44,106.92],[10.68,107.10]], # Cái Mép–Thị Vải
    [[10.55,106.64],[10.86,106.90]], # TP.HCM (Cát Lái/Hiệp Phước)
    [[20.70,106.85],[20.93,107.18]], # Hải Phòng – Lạch Huyện
    [[16.02,108.15],[16.21,108.30]], # Đà Nẵng (Tiên Sa)
    [[13.70,109.18],[13.85,109.30]], # Quy Nhơn
    [[1.18,103.60],[1.33,103.92]],   # Singapore
    [[2.93,101.27],[3.10,101.39]],   # Port Klang
    [[1.30,103.45],[1.43,103.63]],   # Tanjung Pelepas (PTP)
    [[13.00,100.82],[13.14,100.93]], # Laem Chabang
    [[22.25,113.90],[22.40,114.29]], # Hong Kong
    [[22.50,114.18],[22.62,114.32]], # Yantian
    [[22.42,113.80],[22.56,113.98]], # Shekou
    [[22.68,113.44],[22.90,113.70]]  # Nansha
]


# Lọc các message mang thông tin vị trí để giảm tải
FILTER_MESSAGE_TYPES = [
    "PositionReport",
]

# Nếu muốn theo MMSI cụ thể, thêm vào list (tối đa 50 theo docs)
FILTER_MMSI = []  # ví dụ: ["574123000", "563012345"]

# Tên file CSV xuất dữ liệu
CSV_PATH = "ais_positions.csv"

# ======= HÀM HỖ TRỢ =======
def ensure_csv_header(path: str):
    header = [
        "time_utc",
        "message_type",
        "mmsi",
        "ship_name",
        "latitude",
        "longitude",
        "sog",
        "cog",
        "raw_json",
    ]
    file_exists = os.path.exists(path)
    with open(path, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if not file_exists:
            w.writerow(header)

def extract_lat_lon(ais_msg: dict, message_type: str, metadata: dict):
    """
    Trả về (lat, lon) ưu tiên từ Metadata nếu có (docs cho biết server gửi kèm),
    nếu không có thì fallback vào các field trong từng message type.

    LongRangeAisBroadcastMessage đôi khi có Latitude1/2, Longitude1/2 → lấy trung bình.
    """
    lat = metadata.get("latitude") or metadata.get("Latitude")
    lon = metadata.get("longitude") or metadata.get("Longitude")

    if lat is not None and lon is not None:
        return float(lat), float(lon)

    # Fallback theo từng message
    m = ais_msg or {}

    # Common fields
    for la_key, lo_key in [
        ("Latitude", "Longitude"),
        ("lat", "lon"),
    ]:
        if la_key in m and lo_key in m:
            return float(m[la_key]), float(m[lo_key])

    # Long range: có cặp 1/2 → lấy trung bình nếu có
    if message_type == "LongRangeAisBroadcastMessage":
        lat_candidates = [m.get("Latitude1"), m.get("Latitude2")]
        lon_candidates = [float(x) for x in lon_candidates if isinstance(x, (int, float))]
        lat_nums = [float(x) for x in lat_candidates if isinstance(x, (int, float))]
        lon_nums = [float(x) for x in lon_candidates if isinstance(x, (int, float))]
        if lat_nums and lon_nums:
            return sum(lat_nums) / len(lat_nums), sum(lon_nums) / len(lon_nums)

    return None, None

def extract_sog_cog(ais_msg: dict):
    sog = ais_msg.get("Sog") or ais_msg.get("sog")
    cog = ais_msg.get("Cog") or ais_msg.get("cog")
    # Một số message dùng số nguyên, một số double; normalize sang float nếu có
    try:
        sog = float(sog) if sog is not None else None
    except (ValueError, TypeError):
        sog = None
    try:
        cog = float(cog) if cog is not None else None
    except (ValueError, TypeError):
        cog = None
    return sog, cog

def extract_mmsi(metadata: dict, ais_msg: dict):
    # Thông thường MMSI trong MetaData.MMSI; đôi khi trong message là UserID
    mmsi = metadata.get("MMSI")
    if not mmsi:
        mmsi = ais_msg.get("UserID")
    return str(mmsi) if mmsi is not None else None

def extract_ship_name(metadata: dict):
    # MetaData thường có 'ShipName' nếu server biết (theo docs)
    return metadata.get("ShipName")

def pretty_ts():
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S.%f %Z")

# ======= CORE =======
async def run_stream():
    api_key = os.getenv("AISSTREAM_API_KEY")
    if not api_key:
        raise RuntimeError("Thiếu biến môi trường AISSTREAM_API_KEY")

    ensure_csv_header(CSV_PATH)

    backoff = 1
    while True:
        try:
            async with websockets.connect(AIS_URL, max_queue=None) as ws:
                # Theo docs: Gửi subscription trong vòng 3 giây sau khi kết nối
                sub_msg = {
                    "APIKey": api_key,
                    "BoundingBoxes": PORT_BBOXES,
                    # Có thể gửi nhiều bbox: [ [sw, ne], [sw2, ne2], ... ]
                    "FiltersShipMMSI": FILTER_MMSI or None,
                    "FilterMessageTypes": FILTER_MESSAGE_TYPES,
                }
                # Loại bỏ None để tránh key rỗng
                sub_msg = {k: v for k, v in sub_msg.items() if v is not None}
                await ws.send(json.dumps(sub_msg))

                print(f"[{pretty_ts()}] Subscribed. Waiting for messages…")

                backoff = 1  # reset backoff khi OK

                async for raw in ws:
                    try:
                        data = json.loads(raw)
                        msg_type = data.get("MessageType")
                        metadata = data.get("MetaData", {}) or data.get("Metadata", {})  # docs dùng MetaData
                        # Dạng: {"Message": {"PositionReport": {...}}}
                        msg_container = data.get("Message", {})
                        ais_msg = None
                        if msg_type and isinstance(msg_container, dict):
                            ais_msg = msg_container.get(msg_type)

                        if not msg_type or not isinstance(ais_msg, dict):
                            continue  # bỏ những message lạ

                        # Lấy MMSI, tên, vị trí, SOG/COG
                        mmsi = extract_mmsi(metadata, ais_msg)
                        ship_name = extract_ship_name(metadata)
                        lat, lon = extract_lat_lon(ais_msg, msg_type, metadata)
                        sog, cog = extract_sog_cog(ais_msg)

                        # Chỉ ghi khi có lat/lon
                        if lat is not None and lon is not None:
                            row = [
                                metadata.get("time_utc") or pretty_ts(),
                                msg_type,
                                mmsi,
                                ship_name,
                                f"{lat:.6f}",
                                f"{lon:.6f}",
                                "" if sog is None else sog,
                                "" if cog is None else cog,
                                json.dumps(data, ensure_ascii=False),
                            ]
                            with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
                                csv.writer(f).writerow(row)

                            # In gọn ra màn hình
                            print(
                                f"[{metadata.get('time_utc') or pretty_ts()}] "
                                f"{msg_type} | MMSI:{mmsi} | {ship_name or '-'} | "
                                f"({lat:.5f}, {lon:.5f}) | SOG:{sog} | COG:{cog}"
                            )

                    except Exception as e:
                        # Không kill stream vì 1 bản tin lỗi
                        print(f"[{pretty_ts()}] Parse error: {e}")

        except Exception as e:
            print(f"[{pretty_ts()}] Disconnected/error: {e}. Reconnecting in {backoff}s…")
            await asyncio.sleep(backoff)
            backoff = min(backoff * 2, 60)  # exponential backoff tối đa 60s

# Run the stream directly
await run_stream()

[2025-11-01 07:45:57.821942 UTC] Subscribed. Waiting for messages…
[2025-11-01 07:45:58.354184978 +0000 UTC] PositionReport | MMSI:563079500 | CRESCENT RIVER       | (1.23966, 103.76646) | SOG:0.3 | COG:194.8
[2025-11-01 07:45:58.384457223 +0000 UTC] PositionReport | MMSI:477995677 | - | (22.31597, 114.14849) | SOG:None | COG:316.2
[2025-11-01 07:45:58.435357726 +0000 UTC] PositionReport | MMSI:563130300 | EPITOME              | (1.20375, 103.81815) | SOG:7.2 | COG:105.0
[2025-11-01 07:45:58.48634714 +0000 UTC] PositionReport | MMSI:563002940 | WAVEMASTER 6 | (1.26221, 103.81940) | SOG:None | COG:152.3
[2025-11-01 07:45:58.571701147 +0000 UTC] PositionReport | MMSI:477716100 | EVER OASIS           | (22.32633, 114.12594) | SOG:3.3 | COG:248.3
[2025-11-01 07:45:59.026499004 +0000 UTC] PositionReport | MMSI:477997154 | SUN HU               | (22.31736, 114.15050) | SOG:None | COG:210.1
[2025-11-01 07:45:59.12647107 +0000 UTC] PositionReport | MMSI:533133124 | FM BREEZE            | (1.23

CancelledError: 

# ============================================================
# 📊 PORT CONGESTION SNAPSHOT BUILDER
# Chạy ô này bất kỳ lúc nào để tạo file Excel từ dữ liệu AIS
# ============================================================

In [13]:


import os, math, pandas as pd, numpy as np

# ---- Đường dẫn dữ liệu ----
CANDIDATES = [
    "ais_SG.csv","ais_OTHERS.csv","ais_positions.csv",
    "/mnt/data/ais_SG.csv","/mnt/data/ais_OTHERS.csv","/mnt/data/ais_positions.csv"
]
OUT_XLSX = "/mnt/data/port_congestion_report.xlsx"

# ---- Danh sách 13 cảng ----
PORTS = [
    ("Cai Mep - Thi Vai", [[10.44,106.92],[10.68,107.10]], (10.56,107.02), 12),
    ("TP.HCM (Cat Lai/Hiep Phuoc)", [[10.55,106.64],[10.86,106.90]], (10.73,106.79), 12),
    ("Hai Phong - Lach Huyen", [[20.70,106.85],[20.93,107.18]], (20.83,107.03), 10),
    ("Da Nang (Tien Sa)", [[16.02,108.15],[16.21,108.30]], (16.11,108.24), 8),
    ("Quy Nhon", [[13.70,109.18],[13.85,109.30]], (13.77,109.24), 8),
    ("Singapore (PSA)", [[1.18,103.60],[1.33,103.92]], (1.27,103.80), 15),
    ("Port Klang", [[2.93,101.27],[3.10,101.39]], (3.01,101.33), 12),
    ("PTP (Tanjung Pelepas)", [[1.30,103.45],[1.43,103.63]], (1.36,103.55), 12),
    ("Laem Chabang", [[13.00,100.82],[13.14,100.93]], (13.07,100.88), 10),
    ("Hong Kong", [[22.25,113.90],[22.40,114.29]], (22.32,114.12), 12),
    ("Yantian", [[22.50,114.18],[22.62,114.32]], (22.56,114.25), 10),
    ("Shekou/Chiwan/Mawan", [[22.42,113.80],[22.56,113.98]], (22.49,113.90), 10),
    ("Nansha (Guangzhou)", [[22.68,113.44],[22.90,113.70]], (22.80,113.57), 12),
]

# ---- Hàm phụ trợ ----
def in_bbox(lat, lon, bbox):
    (lat0, lon0), (lat1, lon1) = bbox
    return (lat0 <= lat <= lat1) and (lon0 <= lon <= lon1)

def haversine_nm(lat1, lon1, lat2, lon2):
    R = 6371.0088
    dlat, dlon = map(math.radians, [lat2-lat1, lon2-lon1])
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2 * R * math.asin(math.sqrt(a)) * 0.539957  # km → nm

def tag_port_zone(lat, lon):
    for name, bbox, center, wait_nm in PORTS:
        if in_bbox(lat, lon, bbox):
            return name, "in_berth"
        if haversine_nm(lat, lon, center[0], center[1]) <= wait_nm:
            return name, "waiting"
    return None, None

# ---- Đọc dữ liệu AIS ----
dfs = []
for f in CANDIDATES:
    if os.path.exists(f):
        try:
            dfs.append(pd.read_csv(f))
        except Exception:
            pass

if not dfs:
    print("⚠️ Không tìm thấy file CSV AIS — hãy chạy ô stream trước.")
    df = pd.DataFrame()
else:
    df = pd.concat(dfs, ignore_index=True)

if not df.empty:
    for c in ["latitude","longitude","sog","cog"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df["ts"] = pd.to_datetime(df.get("time_utc", pd.Timestamp.utcnow()), errors="coerce", utc=True)
    df = df.dropna(subset=["latitude","longitude","ts"])
    if "message_type" in df.columns:
        df = df[df["message_type"].astype(str).str.contains("PositionReport", na=False)]
else:
    print("⚠️ File rỗng, không có dữ liệu để xử lý.")

# ---- Tag cảng & lọc container-like ----
if not df.empty:
    df["mmsi"] = df["mmsi"].astype(str)
    df[["port","zone"]] = df.apply(lambda r: pd.Series(tag_port_zone(r["latitude"], r["longitude"])), axis=1)
    df = df.dropna(subset=["port"])

    hints = ["MAERSK"," MSC","CMA","EVERGREEN","ONE "," OOCL","COSCO","YANG MING","HAPAG","ZIM","WAN HAI"," PIL "]
    def is_container_like(r):
        name = str(r.get("ship_name") or "").upper()
        return any(h in name for h in hints)
    df = df[df.apply(is_container_like, axis=1)]

# ---- Tính dwell & trạng thái ----
if not df.empty:
    df = df.sort_values(["port","mmsi","ts"])
    df["in_berth_flag"] = ((df["zone"]=="in_berth") & (df["sog"].fillna(0)<0.5)).astype(int)

    def dwell_calc(g):
        g["block"] = (g["in_berth_flag"].shift(fill_value=0) != g["in_berth_flag"]).cumsum()
        if g.iloc[-1]["in_berth_flag"]==1:
            blk = g[g["block"]==g.iloc[-1]["block"]]
            dwell = (blk["ts"].iloc[-1]-blk["ts"].iloc[0]).total_seconds()/3600
        else:
            dwell = 0.0
        g["dwell_hours_now"] = dwell
        return g
    snap = df.groupby(["port","mmsi"], group_keys=False).apply(dwell_calc).groupby(["port","mmsi"], as_index=False).tail(1)

    now = pd.Timestamp.utcnow()
    def rollup(g):
        inb = g[(g["zone"]=="in_berth") & (g["sog"].fillna(0)<0.5)]
        wait = g[(g["zone"]=="waiting") & (g["sog"].fillna(1)<1.0)]
        return pd.Series({
            "TimeUTC": now,
            "InBerth": inb["mmsi"].nunique(),
            "Waiting": wait["mmsi"].nunique(),
            "MedianBerthDwellH": inb["dwell_hours_now"].median() if len(inb)>0 else 0,
            "MaxBerthDwellH": inb["dwell_hours_now"].max() if len(inb)>0 else 0
        })
    summary = snap.groupby("port").apply(rollup).reset_index()

    # ---- Đánh giá “kẹt” ----
    RULE = {"min_inberth":6,"min_waiting":8,"min_median_dwell_h":2.0}
    def flag(r):
        reasons=[]
        if r["InBerth"]>=RULE["min_inberth"]: reasons.append(f"InBerth≥{RULE['min_inberth']}")
        if r["Waiting"]>=RULE["min_waiting"]: reasons.append(f"Waiting≥{RULE['min_waiting']}")
        if r["MedianBerthDwellH"]>=RULE["min_median_dwell_h"]: reasons.append(f"MedianDwell≥{RULE['min_median_dwell_h']}h")
        return "Congested" if reasons else "Normal","; ".join(reasons) if reasons else "—"
    summary[["Status","Reason"]] = summary.apply(flag, axis=1)
    summary = summary.sort_values(["Status","InBerth","Waiting"], ascending=[True,False,False])

    # ---- Xuất Excel ----
    with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd hh:mm") as w:
        summary.to_excel(w, "Summary", index=False)
        snap.rename(columns={"ship_name":"ShipName","dwell_hours_now":"DwellH","ts":"LastSeenUTC"})\
            .to_excel(w, "Details", index=False)
        notes = []
        for _, r in summary.iterrows():
            if r["Status"]=="Congested":
                notes.append(f"{r['port']}: KẸT — InBerth={r['InBerth']}, Waiting={r['Waiting']}, Median≈{r['MedianBerthDwellH']:.1f}h")
            else:
                notes.append(f"{r['port']}: bình thường — InBerth={r['InBerth']}, Waiting={r['Waiting']}")
        w.book.add_worksheet("Analysis").write(0,0,"\n".join(notes))

    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("Port Congestion Summary", summary)

    print("✅ Đã tạo file Excel:", OUT_XLSX)
else:
    print("⚠️ Không có dữ liệu hợp lệ để tạo bảng.")


/tmp/ipython-input-429133685.py:65: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ts"] = pd.to_datetime(df.get("time_utc", pd.Timestamp.utcnow()), errors="coerce", utc=True)


⚠️ Không có dữ liệu hợp lệ để tạo bảng.
